In [1]:
import torch
import random
import torch.backends.cudnn as cudnn
import numpy as np

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

In [2]:
import torch
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader

In [3]:
torch.manual_seed(0)

transf = tr.Compose(
    [tr.Resize((32, 32)),
     tr.RandomHorizontalFlip(),
     tr.RandomRotation(10),
     tr.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
     tr.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
     tr.ToTensor(),
     tr.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data/', train=False, download=True, transform=transf)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 *5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [6]:
for epoch in range(30):   # 데이터셋을 수차례 반복합니다.

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data

        # 변화도(Gradient) 매개변수를 0으로 만들고
        # 옵티마이저 초기화
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.230
[1,  4000] loss: 1.929
[1,  6000] loss: 1.790
[1,  8000] loss: 1.680
[1, 10000] loss: 1.632
[1, 12000] loss: 1.598
[2,  2000] loss: 1.549
[2,  4000] loss: 1.539
[2,  6000] loss: 1.518
[2,  8000] loss: 1.490
[2, 10000] loss: 1.475
[2, 12000] loss: 1.457
[3,  2000] loss: 1.420
[3,  4000] loss: 1.405
[3,  6000] loss: 1.413
[3,  8000] loss: 1.399
[3, 10000] loss: 1.376
[3, 12000] loss: 1.382
[4,  2000] loss: 1.332
[4,  4000] loss: 1.352
[4,  6000] loss: 1.340
[4,  8000] loss: 1.339
[4, 10000] loss: 1.327
[4, 12000] loss: 1.332
[5,  2000] loss: 1.290
[5,  4000] loss: 1.302
[5,  6000] loss: 1.322
[5,  8000] loss: 1.291
[5, 10000] loss: 1.278
[5, 12000] loss: 1.307
[6,  2000] loss: 1.267
[6,  4000] loss: 1.265
[6,  6000] loss: 1.273
[6,  8000] loss: 1.261
[6, 10000] loss: 1.258
[6, 12000] loss: 1.248
[7,  2000] loss: 1.230
[7,  4000] loss: 1.229
[7,  6000] loss: 1.236
[7,  8000] loss: 1.242
[7, 10000] loss: 1.237
[7, 12000] loss: 1.217
[8,  2000] loss: 1.200
[8,  4000] 

In [7]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [8]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [9]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

Predicted:  plane truck deer  deer 


In [10]:
correct = 0
total = 0
# 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요가 없습니다
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # 신경망에 이미지를 통과시켜 출력을 계산합니다
        outputs = net(images)
        # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택하겠습니다
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 61 %


In [11]:
# 각 분류(class)에 대한 예측값 계산을 위해 준비
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# 변화도는 여전히 필요하지 않습니다
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # 각 분류별로 올바른 예측 수를 모읍니다
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# 각 분류별 정확도(accuracy)를 출력합니다
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 68.0 %
Accuracy for class: car   is 80.2 %
Accuracy for class: bird  is 46.5 %
Accuracy for class: cat   is 45.3 %
Accuracy for class: deer  is 49.4 %
Accuracy for class: dog   is 53.2 %
Accuracy for class: frog  is 69.1 %
Accuracy for class: horse is 66.4 %
Accuracy for class: ship  is 69.6 %
Accuracy for class: truck is 64.5 %
